In [1]:
import napari
import numpy as np
from pathlib import Path
from tifffile import imread

from napari_cellseg3d.dev_scripts import evaluate_labels as eval
from napari_cellseg3d.utils import resize
from napari_cellseg3d.code_models.model_instance_seg import (
    binary_connected,
    binary_watershed,
    voronoi_otsu,
    to_semantic,
)

In [2]:
viewer = napari.Viewer()

In [3]:
im_path = Path("C:/Users/Cyril/Desktop/test/instance_test")
prediction_path = str(im_path / "trailmap_ms/trailmap_pred.tif")
gt_labels_path = str(im_path / "labels_relabel_unique.tif")

prediction = imread(prediction_path)
gt_labels = imread(gt_labels_path)

zoom = (1 / 5, 1, 1)
# prediction_resized = resize(prediction, zoom)
prediction_resized = prediction # for trailmap
gt_labels_resized = resize(gt_labels, zoom)


viewer.add_image(prediction_resized, name="pred", colormap="inferno")
viewer.add_labels(gt_labels_resized, name="gt")

In the future `np.bool` will be defined as the corresponding NumPy scalar.
In the future `np.bool` will be defined as the corresponding NumPy scalar.
In the future `np.bool` will be defined as the corresponding NumPy scalar.
In the future `np.bool` will be defined as the corresponding NumPy scalar.


<Labels layer 'gt' at 0x256d14f7610>

In [4]:
from napari_cellseg3d.utils import dice_coeff

semantic_gt = to_semantic(gt_labels_resized.copy())
semantic_pred = to_semantic(prediction_resized.copy())

viewer.add_image(semantic_gt, colormap='bop blue')
viewer.add_image(semantic_pred, colormap='red')

dice_coeff(
    semantic_gt,
    prediction_resized
)

0.7538125057831502

In [5]:
from napari_cellseg3d.dev_scripts.correct_labels import relabel

# gt_corrected = relabel(prediction_path, gt_labels_path, go_fast=False)

In [6]:
# eval.evaluate_model_performance(gt_labels_resized, gt_labels_resized)

In [7]:
print(prediction_resized.shape)
print(gt_labels_resized.shape)
print(np.unique(gt_labels_resized).shape[0])

(25, 64, 64)
(25, 64, 64)
125


In [8]:
connected = binary_connected(prediction_resized, thres_small=2)
viewer.add_labels(connected, name="connected")

<Labels layer 'connected' at 0x256c5a416d0>

In [9]:
eval.evaluate_model_performance(gt_labels_resized, connected)

2023-03-24 14:23:13,590 - Mapping labels...


100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 103/103 [00:00<00:00, 2689.96it/s]

2023-03-24 14:23:13,631 - Calculating the number of neurons not found...
2023-03-24 14:23:13,634 - Percent of non-fused neurons found: 50.40%
2023-03-24 14:23:13,635 - Percent of fused neurons found: 36.00%
2023-03-24 14:23:13,635 - Overall percent of neurons found: 86.40%


(63,
 45,
 16,
 16,
 0.819027731148306,
 0.8401649108992161,
 0.83609908334452,
 0.8066092803671974,
 0.98)

In [10]:
watershed = binary_watershed(
    prediction_resized, thres_small=2, rem_seed_thres=1
)
viewer.add_labels(watershed)
eval.evaluate_model_performance(gt_labels_resized, watershed)

2023-03-24 14:23:13,732 - Mapping labels...


100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 101/101 [00:00<00:00, 5221.10it/s]

2023-03-24 14:23:13,761 - Calculating the number of neurons not found...
2023-03-24 14:23:13,774 - Percent of non-fused neurons found: 61.60%
2023-03-24 14:23:13,775 - Percent of fused neurons found: 27.20%
2023-03-24 14:23:13,776 - Overall percent of neurons found: 88.80%


(77,
 34,
 13,
 9,
 0.728461197681457,
 0.8885669859686413,
 0.8950588507577087,
 0.7472814623489069,
 0.878614359974009)

In [11]:
voronoi = voronoi_otsu(prediction_resized, 0.6, outline_sigma=0.7)

from skimage.morphology import remove_small_objects

voronoi = remove_small_objects(voronoi, 2)
viewer.add_labels(voronoi)
voronoi.shape

(25, 64, 64)

In [12]:
gt_labels_resized.dtype

dtype('int64')

In [13]:
# np.unique(voronoi, return_counts=True)

In [14]:
# np.unique(gt_labels_resized, return_counts=True)

In [15]:
eval.evaluate_model_performance(gt_labels_resized, voronoi)

2023-03-24 14:23:14,241 - Mapping labels...


100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 119/119 [00:00<00:00, 2376.22it/s]

2023-03-24 14:23:14,301 - Calculating the number of neurons not found...
2023-03-24 14:23:14,303 - Percent of non-fused neurons found: 81.60%
2023-03-24 14:23:14,304 - Percent of fused neurons found: 6.40%
2023-03-24 14:23:14,305 - Overall percent of neurons found: 88.00%


(102,
 8,
 14,
 16,
 0.708505702558253,
 0.8832633585884945,
 0.9759871495093808,
 0.6670483272595948,
 0.8653680990771155)

In [16]:
# eval.evaluate_model_performance(gt_labels_resized, voronoi)